# Running Azure Cosmos Gremlin

I've built a lot of my own helper functions to make queries and manipulate data. I'll document them here

It isim. 

First, I'm only using `nest_asyncio` to run the queries in cells. This is a requirement of how gremlinpython manages requests. 

In [1]:
import sys
import pandas as pd
sys.path.append('..')
import helpers.dbquery as db
# import helpers.functions as f
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
db.run_query()

[8059]

the method `run_query` takes a query string and returns the json response. It manages the opening and closing of objects so you don't have to. 

In [3]:
res = db.run_query("g.V().hasLabel('system').has('username','userbill').limit(4).in().inE('orbits').limit(2)")
res

[{'id': '3fb17376-5233-4063-847b-daf5747500f6',
  'label': 'orbits',
  'type': 'edge',
  'inVLabel': 'star',
  'outVLabel': 'planet',
  'inV': 'f378ac7b-ac56-4c50-9eda-c2cd5426a648',
  'outV': 'b1295297-dd7a-45a6-b8b0-f53556dc39c8',
  'properties': {'username': 'userbill'}},
 {'id': '1984978a-d700-40ea-b392-cb4ceb08fc07',
  'label': 'orbits',
  'type': 'edge',
  'inVLabel': 'star',
  'outVLabel': 'planet',
  'inV': 'f378ac7b-ac56-4c50-9eda-c2cd5426a648',
  'outV': 'a3c6c29c-b940-4f5b-b626-33c225b1b131',
  'properties': {'username': 'userbill'}}]

This makes a great `dataframe`

In [4]:
df = pd.DataFrame(db.run_query("g.V().hasLabel('system').has('username','userbill').in().inE('orbits')"))
df.head()

,id,label,type,inVLabel,outVLabel,inV,outV,properties
0,eaf7c9f0-85ce-4df9-b485-1c265a2ed629,orbits,edge,star,planet,c68465f8-6a4c-4395-bdca-490418bb92e9,e9e3acdf-2499-42d2-9c0a-28d5ea156a31,{'username': 'userbill'}
1,2bfc61d4-4aab-4e94-9d07-e1d6310bd909,orbits,edge,star,planet,c68465f8-6a4c-4395-bdca-490418bb92e9,81d22fc0-94bf-4cff-8e50-f13918036e83,{'username': 'userbill'}
2,b7514407-16e6-472f-9931-30b197d7477c,orbits,edge,star,planet,c68465f8-6a4c-4395-bdca-490418bb92e9,32020fe0-1a7c-4c5c-80b7-310463f02fcb,{'username': 'userbill'}
3,8032638d-6dbd-4931-8259-f395a8a3bf35,orbits,edge,star,planet,c68465f8-6a4c-4395-bdca-490418bb92e9,9ad9d290-b3a3-4ddf-9de6-c335e5de8862,{'username': 'userbill'}
4,fc8d4a07-d15c-4187-a3e6-80dfc91a6824,orbits,edge,star,planet,c68465f8-6a4c-4395-bdca-490418bb92e9,fe6bbd75-6097-48af-bb64-d7c66e951107,{'username': 'userbill'}


## Adding Vertices and Edges

I've added some functions that create the `g.addV()` command with all of the properties I want. This also forces some properties like `username` and `objtype` so I don't have to think about them. The function `create_vertex` just returns the string.

**Note:** I'm using objtype as an extra lable as I found the query to be slightly cheaper when querying that value (e.g. `valueMap()`)than returning the actual lable maps (e.g. `valueMap(true)`).

In the game, I'm adding `username` for the user's account so that I can easily filter data for that user. However in notebooks I've replaced the user account with `notebook` so that I can cleanup notebook runs easily.  This `username` is forced to `notebook`, so I can quickly cleanup. 

The node must include a label

In [5]:
nodes = [{"label":"example","objid":db.uuid(),"property1":"foo","property2":"bar"},
        {"label":"example","objid":db.uuid(),"property1":"foo","property2":"bar"}]
[db.create_vertex(node) for node in nodes]

["g.addV('example').property('objid','6142915927627').property('property1','foo').property('property2','foo').property('username','notebook').property('objtype','example')",
 "g.addV('example').property('objid','8364060463296').property('property1','foo').property('property2','foo').property('username','notebook').property('objtype','example')"]

edges reuqure a `node1` and a `node2` object.

In [6]:
edge = {"label":"isExample","node1":nodes[0]["objid"],"node2":nodes[1]["objid"]}
db.create_edge(edge)

"g.V().has('objid','6142915927627').addE('isExample').property('username','notebook').to(g.V().has('objid','8364060463296'))"

The method `upload_data` requires a dict of both edges and nodes

In [8]:
data = {'nodes':nodes,'edges':[edge]}
db.upload_data(data)

Once you have the values you can query them back, to confirm they are in the API. 

In [11]:
res = db.run_query("g.V().has('username','notebook').valueMap()")
res

[{'objid': ['8064048771781'],
  'property1': ['foo'],
  'property2': ['foo'],
  'username': ['notebook'],
  'objtype': ['example']},
 {'objid': ['2691728947417'],
  'property1': ['foo'],
  'property2': ['foo'],
  'username': ['notebook'],
  'objtype': ['example']},
 {'objid': ['6142915927627'],
  'property1': ['foo'],
  'property2': ['foo'],
  'username': ['notebook'],
  'objtype': ['example']},
 {'objid': ['8364060463296'],
  'property1': ['foo'],
  'property2': ['foo'],
  'username': ['notebook'],
  'objtype': ['example']}]

## Time


In [77]:
res = db.run_query("g.V().hasLabel('time').valueMap()")
time = [db.clean_node(n) for n in res][0]
time

{'currentTime': 11086,
 'updatedFrom': 'azfunction',
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

## Actions and Jobs


In [45]:
actions_query = """
    g.E().haslabel('takingAction').has('status','pending').as('job')
        .outV().as('agent').path().by(valueMap())
"""

A traversal has a much more complicated shape that a simply list-of-nodes.

In [81]:
actions = db.run_query(actions_query)

In [82]:

act = []
for action in actions:
    lab = {}
    for itr,itm in enumerate(action['labels']):
        lab[itm[0]] = db.clean_node(action['objects'][itr])
    act.append(lab)

act


[{'job': {'status': 'pending',
   'actionType': 'patriot_education',
   'weight': '11090',
   'username': 'notebook',
   'name': 'takingAction',
   'applies_to': 'pop',
   'effort': '10',
   'requires_attr': 'faction_loyalty;.1',
   'requirement_description': 'faction loyalty must be > .1',
   'augments_self_properties': 'faction_loyalty,literacy,aggression;0.05,0.01,0.05',
   'comment': 'expand public education programs with a collective theme, increase literacy and faction loyalty. However, will cause some militaristic tendancies.',
   'objtype': 'action'},
  'agent': {'isIdle': 'false',
   'conformity': 0.35,
   'literacy': 0.638,
   'aggression': 0.514,
   'constitution': 0.793,
   'objid': '1634757778071',
   'health': 0.5,
   'faction_no': 1,
   'name': 'Gorskmer Tha',
   'isInFaction': '9195085456882',
   'industry': 0.6535,
   'wealth': 0.6458,
   'faction_loyalty': 0.4262,
   'username': 'BillmanLocal2',
   'objtype': 'pop',
   'id': '1634757778071'}},
 {'job': {'status': 'pen

In [83]:
len(act)

3

In [84]:
j = act[0]['job']
a = act[0]['agent']


In [85]:
def validate_action(time,a):
    print(f"currentTime: {time['currentTime']}; weight: {a['weight']}")
    if int(a['weight']) < int(time['currentTime']):
        return True
    else:
        return False

validate_action(time,j)

currentTime: 11086; weight: 11090


False

In [69]:
patched_properties = j['augments_self_properties'].split(";")[0].split(",")
patched_values = [float(f) for f in j['augments_self_properties'].split(";")[1].split(",")]

new_a = {}
for itr,itm in enumerate(patched_properties):
    new_a[itm] = round(a[itm]+patched_values[itr], 4)

In [70]:
a

{'isIdle': 'false',
 'conformity': 0.35,
 'literacy': 0.638,
 'aggression': 0.514,
 'constitution': 0.793,
 'objid': '1634757778071',
 'health': 0.5,
 'faction_no': 1,
 'name': 'Gorskmer Tha',
 'isInFaction': '9195085456882',
 'industry': 0.6535,
 'wealth': 0.6458,
 'faction_loyalty': 0.4262,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '1634757778071'}

In [71]:
new_a

{'faction_loyalty': 0.4762, 'literacy': 0.648, 'aggression': 0.564}

In [76]:
base_query = f"g.V().has('objid','{a['objid']}')"
for n in new_a.keys():
    base_query += f".properties('{n}',{new_a[n]})"
    
base_query

"g.V().has('objid','1634757778071').properties('faction_loyalty',0.4762).properties('literacy',0.648).properties('aggression',0.564)"

In [4]:
c = db.get_client()

In [6]:
c

In [8]:
c.available_pool_size

4